In [1]:
from pyspark.sql.functions import to_timestamp,hour,minute,when,col,current_timestamp,date_format,lit,unix_timestamp,expr,abs,to_date,rank,datediff
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator,TrainValidationSplit
from pyspark.ml.regression import LinearRegression,RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import os

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import sagemaker_pyspark
import botocore.session

session = botocore.session.get_session()
# credentials = session.get_credentials()

conf = (SparkConf().set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars())))

In [3]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.config(conf = conf).appName("test").getOrCreate()
spark

In [4]:
rawdata_s3 = 's3a://solarpowerbackend/rawdata/'
processed_s3 = 's3a://solarpowerbackend/processed/'
model_s3 = 's3a://solarpowerbackend/model/'

In [5]:
raw_df = spark.read.parquet(f'{processed_s3}rul_dataset')
raw_df = raw_df.withColumn('TIME',date_format(col('DATE_TIME'),'HH:mm:ss'))
raw_df = raw_df.filter(col('TIME').between('06:00:00','18:15:00'))

In [6]:
raw_df = raw_df.withColumn('DATE',to_date(col('DATE_TIME')))

In [7]:
only_fault_df = raw_df.select('DATE_TIME','FAULT_FLAG','SOURCE_KEY','DATE').filter(col('Fault_Flag')==1)

In [8]:
display(only_fault_df)

DataFrame[DATE_TIME: timestamp, FAULT_FLAG: int, SOURCE_KEY: string, DATE: date]

In [9]:
# rank_window = Window.partitionBy(only_fault_df['SOURCE_KEY'],only_fault_df['DATE']).orderBy(only_fault_df['DATE_TIME'])
# fault_rank_on_day = rank().over(rank_window)

In [10]:
fault_rank_df = only_fault_df.select(col('SOURCE_KEY').alias('FAULT_SOURCE_KEY'),col('DATE_TIME').alias('FAULT_DATE_TIME'))

In [11]:
rul_oncondition = (fault_rank_df['FAULT_DATE_TIME']>=raw_df['DATE_TIME']) & (fault_rank_df['FAULT_SOURCE_KEY'] == raw_df['SOURCE_KEY'])

In [12]:
rul_df = fault_rank_df.join(raw_df,on = rul_oncondition,how = 'inner')

In [13]:
display(rul_df)

DataFrame[FAULT_SOURCE_KEY: string, FAULT_DATE_TIME: timestamp, DATE_TIME: timestamp, AMBIENT_TEMPERATURE: double, MODULE_TEMPERATURE: double, IRRADIATION: double, PLANT_ID: int, SOURCE_KEY: string, DC_POWER: double, AC_POWER: double, DAILY_YIELD: double, TOTAL_YIELD: double, Fault_Flag: int, TIME: string, DATE: date]

In [14]:
rul_df = rul_df.withColumn('RUL',(col('FAULT_DATE_TIME').cast('long') - col('DATE_TIME').cast('long'))/60 - datediff(col('FAULT_DATE_TIME'),col('DATE_TIME'))*47*15)

In [15]:
display(rul_df)

DataFrame[FAULT_SOURCE_KEY: string, FAULT_DATE_TIME: timestamp, DATE_TIME: timestamp, AMBIENT_TEMPERATURE: double, MODULE_TEMPERATURE: double, IRRADIATION: double, PLANT_ID: int, SOURCE_KEY: string, DC_POWER: double, AC_POWER: double, DAILY_YIELD: double, TOTAL_YIELD: double, Fault_Flag: int, TIME: string, DATE: date, RUL: double]

In [16]:
rul_df = rul_df.groupby('SOURCE_KEY','DATE_TIME','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','PLANT_ID','AC_POWER','DC_POWER','DAILY_YIELD','TOTAL_YIELD').min('RUL')
rul_df = rul_df.withColumnRenamed('min(RUL)','RUL')
# rul_df = rul_df.filter(col('RUL')>=1440)

In [17]:
rul_df = rul_df.withColumn('AC_POWER',col('AC_POWER') - col('AC_POWER')/(col('RUL')+1)*2)

In [18]:
rul_df = rul_df.withColumn('ACDC',col('AC_POWER')/(col('DC_POWER')+1))

In [19]:
display(rul_df.sort(['SOURCE_KEY','DATE_TIME']).filter(col('RUL')<100))

DataFrame[SOURCE_KEY: string, DATE_TIME: timestamp, AMBIENT_TEMPERATURE: double, MODULE_TEMPERATURE: double, IRRADIATION: double, PLANT_ID: int, AC_POWER: double, DC_POWER: double, DAILY_YIELD: double, TOTAL_YIELD: double, RUL: double, ACDC: double]

In [20]:

str_ind = StringIndexer().setInputCol('SOURCE_KEY').setOutputCol('SOURCE_KEY_NUM')
ohe = OneHotEncoder().setInputCol('SOURCE_KEY_NUM').setOutputCol('encoded_Source_Key')

In [21]:
index_stages = [str_ind,ohe]
ohe_pipeline = Pipeline(stages = index_stages)
rul_indexed_df = ohe_pipeline.fit(rul_df).transform(rul_df)

In [22]:
independent_features = ['AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','AC_POWER','ACDC','DC_POWER','DAILY_YIELD','encoded_Source_Key']
model_input_vector = VectorAssembler().setInputCols(independent_features).setOutputCol('features')

rul_indexed_df = rul_indexed_df.withColumnRenamed('RUL','label')

In [23]:
rul_indexed_train_df,rul_indexed_test_df = rul_indexed_df.randomSplit([0.8,0.2])

In [24]:
lr_model = LinearRegression().setFeaturesCol('features').setLabelCol('label')
lr_model.setPredictionCol('LR_prediction')
rf_model = RandomForestRegressor().setFeaturesCol('features').setLabelCol('label')
rf_model.setPredictionCol('RF_prediction')


RandomForestRegressor_b67b9cd06a1b

In [25]:
model_training_stages  = [model_input_vector]
rul_pipeline = Pipeline(stages=model_training_stages)
rul_trained_df = rul_pipeline.fit(rul_indexed_train_df).transform(rul_indexed_train_df)

In [26]:
rfparamGrid = (ParamGridBuilder()
             .addGrid(rf_model.maxDepth, [2, 5, 10, 20, 30])
#                .addGrid(rf_model.maxDepth, [2, 5, 10])
             .addGrid(rf_model.maxBins, [10, 20, 40, 80, 100])
#                .addGrid(rf_model.maxBins, [5, 10, 20])
             .addGrid(rf_model.numTrees, [5, 20, 50, 100, 500])
#                .addGrid(rf_model.numTrees, [5, 20, 50])
             .build())
rfevaluator = RegressionEvaluator(predictionCol="RF_prediction", labelCol="label", metricName="rmse")

In [27]:
rfcv = CrossValidator(estimator = rf_model,
                      estimatorParamMaps = rfparamGrid,
                      evaluator = rfevaluator,
                      numFolds = 5)

In [ ]:
rfcvModel = rfcv.fit(rul_trained_df)
print(rfcvModel)

# Use test set here so we can measure the accuracy of our model on new data
rfpredictions = rfcvModel.transform(rul_trained_df)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
print('RMSE:', rfevaluator.evaluate(rfpredictions))

In [28]:


rul_lr_model = lr_model.fit(rul_trained_df)
rul_trained_df = rul_lr_model.transform(rul_trained_df)
print('Linear Regression Error: ' ,str(rul_lr_model.summary.meanAbsoluteError))

Linear Regression Error:  1483.7125275707344


In [32]:
rul_trained_df.head()

Row(SOURCE_KEY='1BY6WEcLGh8j5v7', DATE_TIME=datetime.datetime(2020, 5, 18, 7, 15), AMBIENT_TEMPERATURE=22.19873053333333, MODULE_TEMPERATURE=25.319020466666665, IRRADIATION=0.18703004600000003, PLANT_ID=4135001, AC_POWER=247.4482376761282, DC_POWER=2525.285714, DAILY_YIELD=121.7142857, TOTAL_YIELD=6278771.714, label=2370.0, ACDC=0.09794942682248338, SOURCE_KEY_NUM=4.0, encoded_Source_Key=SparseVector(21, {4: 1.0}), features=SparseVector(28, {0: 22.1987, 1: 25.319, 2: 0.187, 3: 247.4482, 4: 0.0979, 5: 2525.2857, 6: 121.7143, 11: 1.0}), LR_prediction=753.886119193161, RF_prediction=2535.939011025386)

In [35]:
rul_rf_model.write().overwrite().save(f'{model_s3}RF_RUL_Model')

In [37]:
from sagemaker_pyspark import SageMakerModel
from sagemaker_pyspark import EndpointCreationPolicy
from sagemaker_pyspark.transformation.serializers import ProtobufRequestRowSerializer
from sagemaker_pyspark.transformation.deserializers import KMeansProtobufResponseRowDeserializer

attachedModel = SageMakerModel(
    existingEndpointName=ENDPOINT_NAME,
    endpointCreationPolicy=EndpointCreationPolicy.DO_NOT_CREATE,
    endpointInstanceType=None,  # Required
    endpointInitialInstanceCount=None,  # Required
    requestRowSerializer=ProtobufRequestRowSerializer(
        featuresColumnName="features"
    ),  # Optional: already default value
    responseRowDeserializer=KMeansProtobufResponseRowDeserializer(  # Optional: already default values
        distance_to_cluster_column_name="distance_to_cluster",
        closest_cluster_column_name="closest_cluster",
    ),
)

AttributeError: 'RandomForestRegressionModel' object has no attribute 'endpointName'

In [36]:
rul_rf_model = rf_model.fit(rul_trained_df)
rul_trained_df = rul_rf_model.transform(rul_trained_df)
# print('Randomforest Regression Error: ' ,str(rul_rf_model.stages[-1].summary.meanAbsoluteError))

In [ ]:
for i in zip(independent_features,rul_rf_model.featureImportances):
  print(i)

In [ ]:
rul_trained_df.select('SOURCE_KEY').distinct().collect()

In [ ]:
display(rul_trained_df.filter(col('SOURCE_KEY')=='bvBOhCH3iADSZry').select('SOURCE_KEY','DATE_TIME','RF_Prediction','LR_Prediction','label'))

In [ ]:
rul_trained_df = rul_trained_df.withColumn('acdc',col('DC_POWER')/col('AC_POWER'))

In [ ]:
display(rul_trained_df.filter(col('SOURCE_KEY')=='1BY6WEcLGh8j5v7').select('DATE_TIME',*(independent_features[:-1]),'label'))